In [ ]:
import pandas as pd 
import numpy as np 
import requests
import json
import time
import os
import re
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
from dotenv import load_dotenv
load_dotenv()
import tiktoken
from urllib.parse import urlparse

In [156]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin

def get_internal_links(url):
    try:    # Send a request to the website
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
        response = requests.get(url,headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching the URL: {e}")
        return None
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Get the base domain of the provided URL
    base_domain = urlparse(url).netloc

    # Retrieve all the <a> tags with 'href' attribute
    link_tags = soup.find_all('a', href=True)

    internal_links = []
    for tag in link_tags:
        href = tag['href']
        # Exclude PDF files
        if href.lower().endswith('.pdf'):
            continue
        parsed_url = urlparse(href)
        if parsed_url.netloc in ['', base_domain] or href.startswith('/'):
            # Convert relative path to absolute URL
            absolute_url = urljoin(url, href)
            internal_links.append(absolute_url)

    return internal_links

def extract_main_content(url):
    if not url:
        return ""

    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
        response = requests.get(url,headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching the URL: {e}")
        return ""

    soup = BeautifulSoup(response.content, "html.parser")

    # Remove unwanted elements
    for tag in ['head','header', 'footer']:
        elements_to_remove = soup.find_all(tag)
        for elem in elements_to_remove:
            elem.decompose()

    # Extract text from the remaining elements
    text_content = ' '.join(soup.stripped_strings)
    # remove extra spaces and line breaks
    text_content = re.sub(r"\s+", " ", text_content).strip()
    # add spaces between words that are capitalized
    text_content = re.sub(r'(?<!^)(?=[A-Z])', ' ', text_content)
    # remove punctuation
    words = text_content.split()
    # remove single word characters because of middle initials
    filtered_words = [word for word in words if len(word) > 1]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

# Example usage
website_url = "https://www.concordlakeacademy.org/"
internal_links = get_internal_links(website_url)
unique_list = list(set(internal_links))
unique_list = [url for url in unique_list if "javascript" not in url]
print(f'The website url has {len(unique_list)} internal links')

# Print the internal links

The website url has 82 internal links


In [ ]:
sample_url = 'https://www.concordlakeacademy.org/apps/pages/index.jsp?uREC_ID=505949&type=d&pREC_ID=page'
sample_extract=extract_main_content(sample_url)

In [ ]:
sample_extract

In [ ]:
GPT_MODEL = 'text-embedding-ada-002'
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

In [ ]:
# sample size of tokens
num_tokens(sample_extract)

In [ ]:
# filter links that have less than 60 tokens
filtered_links = []
for link in unique_list:
    content = extract_main_content(link)
    token_count = num_tokens(content)
    if token_count >= 60:
        filtered_links.append(link)

print(filtered_links)

In [ ]:
filtered_links

In [ ]:
merge_charter=pd.read_csv('merge_charter_district.csv')

In [ ]:
merge_charter.head(2)

In [ ]:
embed_content_df=merge_charter[['schoolname','board_of_directors_link']].copy(deep=True)

In [146]:
# link parsing
def get_root_domain(url):
    # Parse the URL
    parsed_url = urlparse(url)
    # Return the root domain
    return parsed_url.netloc

def get_filtered_list(url):
    internal_links = get_internal_links(url)
    if internal_links is None:
        return None
    unique_list = list(set(internal_links))
    filtered_links = [url for url in unique_list if "javascript" not in url]
    clean_filtered_links = [link for link in filtered_links if link.startswith('https://') or link.startswith('https://')]
    return clean_filtered_links

In [ ]:
website_url="http://"
internal_links = get_internal_links(website_url)
unique_list = list(set(internal_links))
unique_list = [url for url in unique_list if "javascript" not in url]

In [ ]:
embed_content_df['valid_web'] = embed_content_df['board_of_directors_link'].apply(lambda x: get_root_domain(x))

In [131]:
embed_content_df['filtered_list']=embed_content_df['valid_web'].apply(lambda x: get_filtered_list("http://"+x))

In [150]:
embed_content_df.to_csv('embed_content_df.csv',index=False)

In [157]:
# check list of links if it meets the token count
embed_content_df['tokenize_filtered_list']=embed_content_df['filtered_list'].apply(lambda x: [link for link in x if num_tokens(extract_main_content(link)) >= 60])
#explode the list of links into rows # double check if this works right or if you have to encapulate in a list
#embed_content_df.explode('filtered_list')
#

An error occurred while fetching the URL: 404 Client Error: Not Found for url: https://ncva.k12.com/www.p3campus.com/tipform....=3000&HF=1&Logo=0&X=1&H=0
An error occurred while fetching the URL: 404 Client Error: Not Found for url: https://www.joycharter.org/https:/docs.google.com/document/d/1dtQeKyEMr7U1pF-sc4-2YFUS8I9ZBSnsISy0bVAQijw/edit
An error occurred while fetching the URL: 401 Client Error: Unauthorized for url: https://www.joycharter.org/resources-1


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


An error occurred while fetching the URL: 401 Client Error: Unauthorized for url: https://www.joycharter.org/joyfulnoise
An error occurred while fetching the URL: 404 Client Error: Not Found for url: https://www.voyageracademy.net/covid-19-dashboard/


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


An error occurred while fetching the URL: 404 Client Error: Not Found for url: https://researchtrianglehighschool.org/contribute/


...

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


An error occurred while fetching the URL: 404 Client Error: Not Found for url: https://www.franklinacademy.org/403.shtml/
An error occurred while fetching the URL: 404 Client Error: Not Found for url: https://www.franklinacademy.org/403.shtml/
An error occurred while fetching the URL: 404 Client Error: Not Found for url: https://www.franklinacademy.org/403.shtml/
An error occurred while fetching the URL: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
An error occurred while fetching the URL: 404 Client Error: Not Found for url: https://www.cemcs.org/?page_id=3276
An error occurred while fetching the URL: 404 Client Error: Not Found for url: https://www.cemcs.org/parent-resources/re-opening-plan/


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


An error occurred while fetching the URL: HTTPConnectionPool(host='cart', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001CEFDE4E8F0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
An error occurred while fetching the URL: 429 Client Error: Too Many Requests for url: http://carolinacharter.org/
An error occurred while fetching the URL: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
An error occurred while fetching the URL: HTTPConnectionPool(host='checkout', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001CEFDEA0280>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
An error occurred while fetching the URL: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
An error occurred while fetching th

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [162]:
embed_content_df=embed_content_df.explode('tokenize_filtered_list')

In [177]:
gov_sites =['www.dpi.nc.gov', 'www.abss.k12.nc.us']
embed_content_df['invalid_web'] = embed_content_df['valid_web'].isin(gov_sites).astype(int)

In [178]:
embed_content_df

,schoolname,board_of_directors_link,valid_web,filtered_list,tokenize_filtered_list,invalid_web
0,North Carolina Cyber Academy,https://www.myncca.com/our-school-board,www.myncca.com,"[https://www.myncca.com/technology-tips, https...",https://www.myncca.com/nhs-njhs,0
0,North Carolina Cyber Academy,https://www.myncca.com/our-school-board,www.myncca.com,"[https://www.myncca.com/technology-tips, https...",https://www.myncca.com/about-ncca,0
0,North Carolina Cyber Academy,https://www.myncca.com/our-school-board,www.myncca.com,"[https://www.myncca.com/technology-tips, https...",https://www.myncca.com/engligh-learner-services,0
0,North Carolina Cyber Academy,https://www.myncca.com/our-school-board,www.myncca.com,"[https://www.myncca.com/technology-tips, https...",https://www.myncca.com/learning-coach-center,0
0,North Carolina Cyber Academy,https://www.myncca.com/our-school-board,www.myncca.com,"[https://www.myncca.com/technology-tips, https...",https://www.myncca.com/military-families,0
...,...,...,...,...,...,...
205,Wilson Preparatory Academy,https://www.wilsonpreparatoryacademy.org/apps/...,www.wilsonpreparatoryacademy.org,[http://www.wilsonpreparatoryacademy.org/apps/...,http://www.wilsonpreparatoryacademy.org/apps/p...,0
205,Wilson Preparatory Academy,https://www.wilsonpreparatoryacademy.org/apps/...,www.wilsonpreparatoryacademy.org,[http://www.wilsonpreparatoryacademy.org/apps/...,http://www.wilsonpreparatoryacademy.org/apps/p...,0
205,Wilson Preparatory Academy,https://www.wilsonpreparatoryacademy.org/apps/...,www.wilsonpreparatoryacademy.org,[http://www.wilsonpreparatoryacademy.org/apps/...,http://www.wilsonpreparatoryacademy.org/apps/e...,0
205,Wilson Preparatory Academy,https://www.wilsonpreparatoryacademy.org/apps/...,www.wilsonpreparatoryacademy.org,[http://www.wilsonpreparatoryacademy.org/apps/...,http://www.wilsonpreparatoryacademy.org/apps/c...,0


In [179]:
embed_content_df['website_content'] = embed_content_df.apply(
    lambda row: extract_main_content(row['tokenize_filtered_list']) if row['invalid_web'] == 0 else "", 
    axis=1
)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
c:\Users\chris\.conda\envs\charter\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced w

An error occurred while fetching the URL: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
An error occurred while fetching the URL: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

An error occurred while fetching the URL: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

...

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


An error occurred while fetching the URL: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


An error occurred while fetching the URL: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [180]:
embed_content_df

,schoolname,board_of_directors_link,valid_web,filtered_list,tokenize_filtered_list,invalid_web,website_content
0,North Carolina Cyber Academy,https://www.myncca.com/our-school-board,www.myncca.com,"[https://www.myncca.com/technology-tips, https...",https://www.myncca.com/nhs-njhs,0,top of page National Honor Society The Nationa...
0,North Carolina Cyber Academy,https://www.myncca.com/our-school-board,www.myncca.com,"[https://www.myncca.com/technology-tips, https...",https://www.myncca.com/about-ncca,0,top of page About Cyber Academy A) is one of t...
0,North Carolina Cyber Academy,https://www.myncca.com/our-school-board,www.myncca.com,"[https://www.myncca.com/technology-tips, https...",https://www.myncca.com/engligh-learner-services,0,top of page Multi- Lingual Learner Services En...
0,North Carolina Cyber Academy,https://www.myncca.com/our-school-board,www.myncca.com,"[https://www.myncca.com/technology-tips, https...",https://www.myncca.com/learning-coach-center,0,top of page Learning Coach Center Over the yea...
0,North Carolina Cyber Academy,https://www.myncca.com/our-school-board,www.myncca.com,"[https://www.myncca.com/technology-tips, https...",https://www.myncca.com/military-families,0,top of page Military Family Resources North Ca...
...,...,...,...,...,...,...,...
205,Wilson Preparatory Academy,https://www.wilsonpreparatoryacademy.org/apps/...,www.wilsonpreparatoryacademy.org,[http://www.wilsonpreparatoryacademy.org/apps/...,http://www.wilsonpreparatoryacademy.org/apps/p...,0,Google Translate Skip to main content Skip to ...
205,Wilson Preparatory Academy,https://www.wilsonpreparatoryacademy.org/apps/...,www.wilsonpreparatoryacademy.org,[http://www.wilsonpreparatoryacademy.org/apps/...,http://www.wilsonpreparatoryacademy.org/apps/p...,0,Google Translate Skip to main content Skip to ...
205,Wilson Preparatory Academy,https://www.wilsonpreparatoryacademy.org/apps/...,www.wilsonpreparatoryacademy.org,[http://www.wilsonpreparatoryacademy.org/apps/...,http://www.wilsonpreparatoryacademy.org/apps/e...,0,Today May 2023 Main Calendar Cal Compact Mode ...
205,Wilson Preparatory Academy,https://www.wilsonpreparatoryacademy.org/apps/...,www.wilsonpreparatoryacademy.org,[http://www.wilsonpreparatoryacademy.org/apps/...,http://www.wilsonpreparatoryacademy.org/apps/c...,0,Google Translate Skip to main content Skip to ...


In [188]:
embed_content_df.to_csv('embed_content_df.csv',index=False,escapechar='\\',encoding='utf-8',errors='ignore')

In [190]:
embed_content_df_input = pd.read_csv('embed_content_df.csv')

In [191]:
embed_content_df_input

,schoolname,board_of_directors_link,valid_web,filtered_list,tokenize_filtered_list,invalid_web,website_content
0,North Carolina Cyber Academy,https://www.myncca.com/our-school-board,www.myncca.com,"['https://www.myncca.com/technology-tips', 'ht...",https://www.myncca.com/nhs-njhs,0,top of page National Honor Society The Nationa...
1,North Carolina Cyber Academy,https://www.myncca.com/our-school-board,www.myncca.com,"['https://www.myncca.com/technology-tips', 'ht...",https://www.myncca.com/about-ncca,0,top of page About Cyber Academy A) is one of t...
2,North Carolina Cyber Academy,https://www.myncca.com/our-school-board,www.myncca.com,"['https://www.myncca.com/technology-tips', 'ht...",https://www.myncca.com/engligh-learner-services,0,top of page Multi- Lingual Learner Services En...
3,North Carolina Cyber Academy,https://www.myncca.com/our-school-board,www.myncca.com,"['https://www.myncca.com/technology-tips', 'ht...",https://www.myncca.com/learning-coach-center,0,top of page Learning Coach Center Over the yea...
4,North Carolina Cyber Academy,https://www.myncca.com/our-school-board,www.myncca.com,"['https://www.myncca.com/technology-tips', 'ht...",https://www.myncca.com/military-families,0,top of page Military Family Resources North Ca...
...,...,...,...,...,...,...,...
8570,Wilson Preparatory Academy,https://www.wilsonpreparatoryacademy.org/apps/...,www.wilsonpreparatoryacademy.org,['http://www.wilsonpreparatoryacademy.org/apps...,http://www.wilsonpreparatoryacademy.org/apps/p...,0,Google Translate Skip to main content Skip to ...
8571,Wilson Preparatory Academy,https://www.wilsonpreparatoryacademy.org/apps/...,www.wilsonpreparatoryacademy.org,['http://www.wilsonpreparatoryacademy.org/apps...,http://www.wilsonpreparatoryacademy.org/apps/p...,0,Google Translate Skip to main content Skip to ...
8572,Wilson Preparatory Academy,https://www.wilsonpreparatoryacademy.org/apps/...,www.wilsonpreparatoryacademy.org,['http://www.wilsonpreparatoryacademy.org/apps...,http://www.wilsonpreparatoryacademy.org/apps/e...,0,Today May 2023 Main Calendar Cal Compact Mode ...
8573,Wilson Preparatory Academy,https://www.wilsonpreparatoryacademy.org/apps/...,www.wilsonpreparatoryacademy.org,['http://www.wilsonpreparatoryacademy.org/apps...,http://www.wilsonpreparatoryacademy.org/apps/c...,0,Google Translate Skip to main content Skip to ...


In [ ]:
#df.explode(list('C')) 

,A,B,C
0,"[0, 3, 4]",1,0
0,"[0, 3, 4]",1,3
0,"[0, 3, 4]",1,4
0,"[0, 3, 4]",1,5
0,"[0, 3, 4]",1,6
1,foo,2,4
1,foo,2,5
2,[],3,NaN
3,"[3, 4]",4,d
3,"[3, 4]",4,e
